# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[13, 7]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,1,0.804,32,Kingsloch,13:7,Starbrook,6,0,town,0,0,13,7
NArea,1,0.553,17,none,13:6,Starbrook,6,0,desert,0,0,13,6
SArea,1,0.952,37,none,13:8,Starbrook,6,0,mountain,8,1,13,8
EArea,1,1.029,23,none,14:7,Starbrook,6,0,desert,0,0,14,7
WArea,1,0.855,33,none,12:7,Starbrook,6,0,mountain,0,0,12,7


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

False

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
6:4,4,0.0,6,6:4,8.0,desert,NaN,3.0,Warchild,0,0,0,1.210
7:4,4,0.0,7,7:4,7.0,desert,NaN,3.0,Warchild,0,0,0,0.933
8:4,4,0.0,8,8:4,6.0,desert,NaN,3.0,Warchild,0,0,0,0.731
9:4,4,0.0,9,9:4,7.0,desert,NaN,3.0,Warchild,0,0,0,0.827
12:4,4,0.0,12,12:4,6.0,desert,NaN,6.0,Starbrook,0,1,0,1.156


You can also _mask_ the terrain where the character has no knowledge. 

In [7]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,0.0,0,0:0,2.0,ocean,NaN,NaN,NaN,0,1,0,0.500
1:0,0,0.0,1,1:0,-2.0,ocean,NaN,NaN,NaN,0,1,0,0.882
2:0,0,0.0,2,2:0,1.0,ocean,NaN,NaN,NaN,0,1,0,0.427
3:0,0,0.0,3,3:0,1.0,ocean,NaN,NaN,NaN,0,1,0,0.467
4:0,0,0.0,4,4:0,1.0,ocean,NaN,NaN,NaN,0,1,0,0.759


Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [8]:
[[person.name for person in t.population] for t in world.towns]

[['Monkam', 'Quisassif', 'Xayangoh'],
 ['Pirnab', 'Chirnorb'],
 ['Hallaf', 'Iragordis', 'Olordave'],
 ['Trirnaf', 'Tickich', 'Drornorb', 'Glasasselle', 'Scomattoh'],
 ['Alrpad', 'Heklich'],
 ['Pinoddof', 'Aniladdich', 'Deggid', 'Quosassace'],
 ['Ekoxeb'],
 ['Scoscadle', 'Rogennaf', 'Fewaker'],
 ['Vockrich', 'Milarkunt'],
 ['Theladdottle'],
 ['Scafillion', 'Shelandai', 'Rascundle', 'Thellut'],
 ['Stifarrunt', 'Quappraf'],
 ['Rekillud', 'Ulckow', 'Legsid'],
 ['Pozed', 'Helarkel'],
 ['Nocinnaker'],
 ['Nekkis', 'Fapprif'],
 ['Stefillow'],
 ['Erokillid'],
 ['Urnnadle'],
 ['Pollox', 'Jomendish'],
 ['Pogordeb'],
 ['Grorpun', 'Ultendace', 'Detoldion'],
 ['Bolanda'],
 ['Thalandep', 'Maharkea'],
 ['Laggnan']]

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [9]:
[[(person.name,person.temperment) for person in t.population if person.temperment>.5] for t in world.towns]

[[],
 [],
 [('Hallaf', 0.9), ('Olordave', 0.83)],
 [('Trirnaf', 0.94), ('Tickich', 0.6), ('Scomattoh', 0.67)],
 [('Alrpad', 0.8)],
 [('Pinoddof', 0.75), ('Deggid', 0.91)],
 [('Ekoxeb', 0.93)],
 [('Scoscadle', 0.64), ('Rogennaf', 0.78)],
 [('Milarkunt', 0.83)],
 [],
 [('Rascundle', 0.65), ('Thellut', 0.98)],
 [('Quappraf', 0.69)],
 [('Ulckow', 0.58), ('Legsid', 0.75)],
 [],
 [],
 [('Nekkis', 0.87), ('Fapprif', 0.97)],
 [('Stefillow', 0.53)],
 [],
 [('Urnnadle', 0.98)],
 [('Pollox', 0.61)],
 [('Pogordeb', 0.61)],
 [('Grorpun', 0.64), ('Ultendace', 0.67), ('Detoldion', 0.98)],
 [('Bolanda', 0.94)],
 [('Thalandep', 0.76), ('Maharkea', 0.51)],
 [('Laggnan', 0.96)]]

You can also get summary stats on people. For example, the average temperment per town.

In [10]:
[(t.name,np.mean([person.temperment for person in t.population])) for t in world.towns]

[('Corerun', 0.3233333333333333),
 ('Hellcombe', 0.15000000000000002),
 ('Hardspring', 0.67),
 ('Earthrise', 0.6),
 ('Rockfort', 0.505),
 ('Kingsfellow', 0.46499999999999997),
 ('Stenchcraft', 0.93),
 ('Puregarden', 0.5),
 ('Hellspring', 0.42),
 ('Kingsloch', 0.26),
 ('Snowborourgh', 0.5525),
 ('Soilpike', 0.5249999999999999),
 ('Rageplace', 0.59),
 ('Lionhenge', 0.37),
 ('Heirwish', 0.23),
 ('Heirhenge', 0.9199999999999999),
 ('Starhenge', 0.53),
 ('Demonscrest', 0.33),
 ('Coretear', 0.98),
 ('Splitcombe', 0.47),
 ('Ratfort', 0.61),
 ('Lilyspring', 0.7633333333333333),
 ('Buckfold', 0.94),
 ('Warrun', 0.635),
 ('Flameforth', 0.96)]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [11]:
where_the_char_has_been = world.df_features.loc[world.df_features['visited']==1]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
13:8,8,0.0,13,13:8,37.0,mountain,NaN,6.0,Starbrook,1,1,8,0.952
13:9,9,0.0,13,13:9,44.0,mountain,NaN,6.0,Starbrook,1,1,7,0.841
12:10,10,0.0,12,12:10,62.0,town,Rageplace,6.0,Starbrook,1,1,5,0.604
13:10,10,0.0,13,13:10,51.0,mountain,NaN,6.0,Starbrook,1,1,6,0.537


In [12]:
nations_where_character_has_been = np.unique(where_the_char_has_been['nation'])
nations_where_character_has_been

array(['Starbrook'], dtype=object)

You can use this to get lists of people who the character knows. 

In [13]:
[[t for t in n.towns] for n in world.nations if n.name in nations_where_character_has_been]

[[town of Kingsloch: population: 1 location: [13,7] founded 3,
  capitol of Rageplace: population: 2 location: [12,10] founded 4,
  town of Ratfort: population: 1 location: [17,10] founded 8]]

In [14]:
[[t for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

[[Theladdottle the Speaker of Kingsloch],
 [Rekillud the Speaker of Rageplace,
  Ulckow the commoner,
  Legsid the Ruler of the nation of Starbrook],
 [Pogordeb the Speaker of Ratfort]]

In [15]:
[[t.get_person_data() for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

[[{'name': 'Theladdottle',
   'role': 'Speaker of Kingsloch',
   'loyalty': 0.5,
   'temperment': 0.26,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'None',
    'None',
    'What a great wedding. So glad the people of [Nation of Starbrook] are our friends.']}],
 [{'name': 'Rekillud',
   'role': 'Speaker of Rageplace',
   'loyalty': 0.5,
   'temperment': 0.44,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'None',
    'None',
    'What a great wedding. So glad the people of [Nation of Starbrook] are our friends.']},
  {'name': 'Ulckow',
   'role': 'commoner',
   'loyalty': 0.5,
   'temperment': 0.58,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'None',
    'None',
    'What a great wedding. So glad the people of [Nation of Starbrook] are our friends.']},
  {'name': 'Legsid',
   'role': 'Ruler of the nation of Starbrook',
   'loyalty': 0.5,
   'temperment': 0.75,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',

In [16]:
masked_world.loc[world.Character.get_location_key()]

y                            7
rainfall                     0
x                           13
key                       13:7
elevation                   32
terrain                   town
feature              Kingsloch
nation number                6
nation               Starbrook
visited                      0
aware                        1
turn_last_visited            0
danger                   0.804
Name: 13:7, dtype: object

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [17]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
12:10,10,0.0,12,12:10,62.0,town,Rageplace,6.0,Starbrook,1,1,5,0.604


## Getting the diplomacy and relationships data for the journal page

In [18]:
rel = w.get_people_where_char_has_visited(world)
rel

[{'town': capitol of Rageplace: population: 2 location: [12,10] founded 4,
  'people': [Rekillud the Speaker of Rageplace,
   Ulckow the commoner,
   Legsid the Ruler of the nation of Starbrook]}]

In [19]:
w.get_relationships_node_map(world)

[{'name': 'Rageplace',
  'children': [{'name': 'Rekillud',
    'role': 'Speaker of Rageplace',
    'temperment': 0.44,
    'loyalty': 0.5},
   {'name': 'Ulckow', 'role': 'commoner', 'temperment': 0.58, 'loyalty': 0.5},
   {'name': 'Legsid',
    'role': 'Ruler of the nation of Starbrook',
    'temperment': 0.75,
    'loyalty': 0.5}]}]